<a href="https://colab.research.google.com/github/nishant2k/bird_classification/blob/master/paddyproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!unzip "/content/drive/My Drive/birds_data.zip"

In [0]:
len(os.listdir("/content/bird_f/train"))

6009

In [0]:
import os
len(os.listdir("/content/bird_f/test"))

1480

In [0]:
"""Importing nc=ecessery libraries"""
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
import random
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

"""Function for building th model"""
def build_model(size, num_classes):
    inputs = Input((size, size, 3))
    backbone = MobileNetV2(input_tensor=inputs, include_top=False, weights="imagenet") #Used mobilenetV2 for better performance of our model.
    backbone.trainable = True
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation="relu")(x)
    
    #x = Dense(1024, activation="relu")(x)
    #x = Dense(1024, activation="relu")(x) # No need to dence the layers because we are using MobileNetV2
    x = Dense(num_classes, activation="softmax")(x)


    model = tf.keras.Model(inputs, x)
    return model
"""Function for reading images"""
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

def parse_data(x, y):
    x = x.decode()

    num_class = 211
    size = 224

    image = read_image(x, size)
    label = [0] * num_class
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)

    return image, label

def tf_parse(x, y):
    x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
    x.set_shape((224, 224, 3))
    y.set_shape((211))
    return x, y

"""Function for taking the image locations and taking out informations and storing them"""
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset


path = "/content/bird_f" # Path of directory containing train,test and labels
train_path = os.path.join(path, "train/*") #path of train dataset
#test_path = os.path.join(path, "test/*")
labels_path = os.path.join(path, "train_labels.csv") #labels of train datasets


labels_df = pd.read_csv(labels_path) #Reading the train label
breed = labels_df["breed"].unique() # Making list of unique elements of breed of birds
print("Number of Breed: ", len(breed))

breed2id = {name: i for i, name in enumerate(breed)} #making dictionary for birds name with index
id2breed = {i: name for i, name in enumerate(breed)} # vice-versa of breed2id
 

ids = glob(train_path) # randomly shuffling and giving the path of train_path
labels = []

#Reading every image and taking their breed
for image_id in ids: 
    image_id = image_id.split("/")[-1].split(".")[0]
    breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
    breed_idx = breed2id[breed_name]
    labels.append(breed_idx)


"""Splitting the data for training and validation"""
train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)


size = 224
num_classes = 211 # no. of breeds of birds
lr = 1e-5 # learning rate of our model
batch = 16
epochs = 20

    ## Building the DL model from the function created
model = build_model(size, num_classes)
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr), metrics=["acc"])
    

    ## Taking the useful data from images
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

    ## Training
callbacks = [
        ModelCheckpoint("model3.h5", verbose=1, save_best_only=True), #saving the model for further use 
        ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-6)
        ]
train_steps = (len(train_x)//batch) + 1
valid_steps = (len(valid_x)//batch) + 1
#Final training the model
model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        validation_data=valid_dataset,
        epochs=epochs,
        callbacks=callbacks)


Number of Breed:  211
Epoch 1/20
301/301 [==============================] - ETA: 0s - loss: 3.9049 - acc: 0.2361
Epoch 00001: val_loss improved from inf to 3.87824, saving model to model3.h5
301/301 [==============================] - 72s 238ms/step - loss: 3.9049 - acc: 0.2361 - val_loss: 3.8782 - val_acc: 0.1473 - lr: 1.0000e-05
Epoch 2/20
301/301 [==============================] - ETA: 0s - loss: 2.6612 - acc: 0.3747
Epoch 00002: val_loss improved from 3.87824 to 3.47205, saving model to model3.h5
301/301 [==============================] - 72s 239ms/step - loss: 2.6612 - acc: 0.3747 - val_loss: 3.4720 - val_acc: 0.2271 - lr: 1.0000e-05
Epoch 3/20
301/301 [==============================] - ETA: 0s - loss: 2.2463 - acc: 0.4606
Epoch 00003: val_loss improved from 3.47205 to 3.16602, saving model to model3.h5
301/301 [==============================] - 72s 238ms/step - loss: 2.2463 - acc: 0.4606 - val_loss: 3.1660 - val_acc: 0.2804 - lr: 1.0000e-05
Epoch 4/20
301/301 [====================

In [0]:
"""Testing of the data and calculating the accuracy for testing data"""
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 


"""Function for reading the images"""
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size)) # Resizing the images for equality 
    image = image / 255.0
    image = image.astype(np.float32)
    return image

path = "/content/bird_f"
#train_path = os.path.join(path, "train/*")
test_path = os.path.join(path, "test/*") #path of test dataset
labels_path2 = os.path.join(path, "test_labels.csv") #path of test_labels


labels_df2 = pd.read_csv(labels_path2)
#breed = labels_df["breed"].unique()
#print("Number of Breed: ", len(breed))

#breed2id = {name: i for i, name in enumerate(breed)}
#id2breed = {i: name for i, name in enumerate(breed)}


"""Calling our model"""
model = tf.keras.models.load_model("model3.h5")

#for i, path in tqdm(enumerate(valid_x[:10])):
l1 = labels_df2["id"] # image ids
l2 = labels_df2["breed"] # corresponding breed of bird

zip1 = list(zip(l1, l2))

random.shuffle((zip1)) # shuffling our data for better results

res1 = list(zip(*zip1))

l1 = list(res1[0])
l2 = list(res1[1])

predicted=[] # predicted array for breed of images
for i in range(len(l1)):
    image = read_image(path + "/" + "test/" + str(l1[i]) + ".png", 224)
    image = np.expand_dims(image, axis=0)
    pred = model.predict(image)[0]
    label_idx = np.argmax(pred)
    breed_name = id2breed[label_idx]
    predicted.append(breed_name)
    
print(accuracy_score(l2,predicted)) # Calculating the accuracy for testing data


0.870945945945946


In [0]:
"""Calculating accuracy of our model for training data"""
#importing necessery libraries
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

#Function for reainf the images
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

path = "/content/bird_f"
#train_path = os.path.join(path, "train/*")
#test_path = os.path.join(path, "test/*") 
labels_path2 = os.path.join(path, "train_labels.csv") #path of train labels


labels_df2 = pd.read_csv(labels_path2)
#breed = labels_df["breed"].unique()
#print("Number of Breed: ", len(breed))

#breed2id = {name: i for i, name in enumerate(breed)}
#id2breed = {i: name for i, name in enumerate(breed)}

"""Calling our model"""
model = tf.keras.models.load_model("model3.h5")

#for i, path in tqdm(enumerate(valid_x[:10])):
l1 = labels_df2["id"] # id of bird
l2 = labels_df2["breed"] #coresponding brred of l1 list

zip1 = list(zip(l1, l2))

random.shuffle((zip1)) #shuffling for unbiasing 
 
res1 = list(zip(*zip1))

l1 = list(res1[0])
l2 = list(res1[1])

predicted=[] # predicated breed for training labels
for i in range(len(l1)):
    image = read_image(path + "/" + "train/" + str(l1[i]) + ".png", 224)
    image = np.expand_dims(image, axis=0)
    pred = model.predict(image)[0]
    label_idx = np.argmax(pred)
    breed_name = id2breed[label_idx]
    predicted.append(breed_name)
    
print(accuracy_score(l2,predicted)) # calculating accuracy of our model for training data


0.9264436678315859
